In [26]:
from crewai import Agent, Crew, Task, LLM

llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434",
)

In [27]:
studyplan_maker = Agent(
    llm=llm,
    role="Curriculum Designer",
    goal=(
        "Design a comprehensive, structured curriculum for {person} on the topic of {topic}, "
        "ensuring that it provides a clear learning path from basic to advanced concepts related to the topic given, "
        "and add key objectives necessary for {person} to master related to the topic"
        "this guideline is for constructing a good material which can be read within a day, so no timeline needed"
    ),
    backstory=(
        "You are a highly skilled Curriculum Designer"
        "Your task is to create personalized learning plans that ensure deep understanding about {topic}"
        "You are focused on providing well-structured, logical, and comprehensive learning experiences. "
        "You strive to help learners master their chosen topics by breaking down concepts, defining clear learning objectives from easy to advanced material"
    ),
    allow_delegation=False,
    verbose=True
)

curriculum_design = Task(
    description=(
        "{person} just reached out with a request to design a curriculum for "
        "the topic of {topic}.\n\n"
        "Use your knowledge and expertise to develop a structured, comprehensive curriculum "
        "that ensures deep understanding of the subject matter.\n"
        "Be sure to structure the curriculum in a logical progression, starting from basic concepts and "
        "gradually advancing to more complex ideas"
    ),
    expected_output=(
        "A detailed, structured curriculum for the topic of {topic}, including:\n"
        "- A clear learning path with well-defined objectives for each stage of learning without any timeline\n"
        "- Breakdown of key topics, with quick notes for each sub topics\n"
        "- Progressively more advanced topics that build on foundational knowledge.\n"
        "Ensure the curriculum addresses all aspects of the topic comprehensively."
    ),
    agent=studyplan_maker
)

In [28]:
material_researcher = Agent(
    llm=llm,
    role="Senior Subject Material Researcher",
    goal="To conduct in-depth research and provide detailed, comprehensive material that you know regarding {topic}"
        "that aligns with the curriculum guidelines provided by the Curriculum Designer.",
    backstory=(
        "You are a highly experienced material researcher tasked with providing a thorough, "
        "well-researched bulk of content on the topic {topic} for {person}.\n\n"
        "Your primary responsibility is to produce material that aligns with the curriculum designed by the Curriculum Designer. "
        "This means your research must follow the prescribed learning path."
        "Your material must cover all necessary concepts, provide clear explanations, examples, and references, "
        "and avoid making assumptions about what the learner already knows.\n\n"
        "You must be meticulous in ensuring that the material is complete, accurate, and well-structured. "
        "Make sure that every key concept is explained fully, without skipping any details."
    ),
    allow_delegation=False,
    verbose=True
)

material_research_task = Task(
    description=(
        "{person} has requested a detailed, informative material on the topic of {topic}.\n\n"
        "You, as a Senior Subject Material Researcher, are responsible for creating a more detailed material "
        "based on the guideline designed by the Curriculum Designer.\n"
        "The material should cover all the key aspects of the topic and provide clear explanations, examples."
        "At this stage you dont need to do any external research or provide material references, just create "
        "the material with all that you currently know regarding the {topic} "
        "Your goal is to ensure that the material is comprehensive, accurate, and fits the learning path outlined by the Curriculum Designer."
    ),
    expected_output=(
        "A thorough and well-researched set of materials on {topic}, including:\n"
        "- Clear, detailed explanations of key concepts and principles.\n"
        "- Examples, illustrations, and case studies that help clarify complex ideas.\n"
        "- A logical structure that follows the curriculum guidelines, ensuring the material aligns with the learning objectives. Show each paragraph is part of which section from the guideline. This is important. \n"
        "- Information at the appropriate depth and level, suitable for the learner's current stage (beginner, intermediate, advanced).\n"
        "Make sure that every concept is explained in full, with no assumptions made, and that the material is comprehensive."
    ),
    context=[curriculum_design],
    agent=material_researcher
)


In [29]:
qna_creator = Agent(
    llm=llm,
    role="Educational Assessor",
    goal="Create relevant and insightful questions for assessing the learner's understanding of the given materials, "
          "engage in a QnA session with the learner to evaluate their knowledge,"
          "and provide detailed, constructive feedback to guide their learning journey further or give notes on what to focus to improve understanding about {topic}",
    backstory=(
        "You are an experienced educational assessor tasked with creating questions that test the learner's understanding of the material provided by the MaterialResearcher."
        "Your role is to create a variety of **assessment questions** along with its answer that range in difficulty, ensuring they cover the key concepts from the material about {topic}.\n\n"
        "You will also suggest further study, exercises, or references if necessary.\n\n"
        "Your main objective is to **help the learner improve** by giving them clear and targeted feedback also place to test their knowledge via self QnA"
        "identifying any gaps in their understanding, and adapting the next set of questions accordingly to reinforce learning."
    ),
    allow_delegation=False,
    verbose=True
)

qna_task = Task(
    description=(
        "{person} has requested that you assess their understanding of the topic {topic}.\n\n"
        "Your task is to create a set of exactly 5 questions based on the material provided by the Material Researcher. "
        "These questions should range in difficulty from basic to advanced and cover all key concepts from the material. "
        "The questions can be in the form of multiple choice, short answer, or applied questions. "
        "For each question, also provide the correct answer to guide {person}'s learning and as for testing {person} understanding of the material. \n\n"
        "After generating the questions, provide clear and targeted feedback for further study, including suggestions for exercises, "
        "additional references, or areas to focus on to improve understanding of the topic."
    ),
    expected_output=(
        "A comprehensive set of exactly 5 assessment questions based on the material provided, ranging from basic to advanced in difficulty. "
        "Each question should be followed by its correct answer.\n"
        "After the questions, provide constructive feedback with guidelines on how to improve understanding, including suggestions for further study, exercises, or resources."
    ),
    context=[material_research_task, curriculum_design],
    agent=qna_creator,
)


In [30]:
crew = Crew(
    agents=[studyplan_maker, material_researcher, qna_creator],
    tasks=[curriculum_design, material_research_task, qna_task],
    verbose=True,
)

result = crew.kickoff(inputs={"person":"Budi", 'topic':'Human Digestive System'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b88c0295-0594-4a5d-8cda-b19af4bf0726                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Curriculum Designer                                                                                     │
│                                                                                                                 │
│  Task: Budi just reached out with a request to design a curriculum for the topic of Human Digestive System.     │
│                                                                                                                 │
│  Use your knowledge and expertise to develop a structured, comprehensive curriculum that ensures deep           │
│  understanding of the subject matter.                                                                           │
│  Be sure to structure the curriculum in a logical progression, starting from basic concepts and gradually       │
│  advancing to more complex ideas                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Curriculum Designer                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Comprehensive Curriculum for Human Digestive System**                                                        │
│                                                                                                                 │
│  **Learning Path:**                                                                                             │
│                                                                                                                 │
│  1. **Introduction to Human Digestive System**                                                                  │
│          * Define what the human digestive system is and its importance.                                        │
│          * Explain the main functions of the digestive system (absorption, elimination, etc.).                  │
│          * Identify the different types of digestion (mechanical, chemical, enzymatic).                         │
│  2. **Anatomy of the Digestive System**                                                                         │
│          * Describe the structure and function of the mouth, esophagus, stomach, small intestine, and large     │
│  intestine.                                                                                                     │
│          * Explain the role of the liver, pancreas, and gallbladder in digestion.                               │
│          * Identify the different layers of the intestinal wall and their functions.                            │
│  3. **Nutrient Absorption**                                                                                     │
│          * Explain how nutrients are absorbed from food into the bloodstream.                                   │
│          * Describe the role of enzymes, acids, and pH levels in nutrient absorption.                           │
│          * Discuss the different types of nutrients (carbohydrates, proteins, fats) and their absorption        │
│  mechanisms.                                                                                                    │
│  4. **Digestive Enzymes and Their Functions**                                                                   │
│          * Introduce the main digestive enzymes (amylase, lipase, trypsin, etc.) and their functions.           │
│          * Explain how these enzymes break down food into smaller molecules.                                    │
│          * Discuss the importance of enzyme balance and regulation in digestion.                                │
│  5. **Gastrointestinal Motility and Peristalsis**                                                               │
│          * Describe the process of peristalsis and its role in moving food through the digestive system.        │
│          * Explain how the enteric nervous system regulates motility.                                           │
│          * Discuss the effects of altered motility on digestion (e.g., constipation, diarrhea).                 │
│  6. **The Small Intestine and Absorption**                                                                      │
│          * Describe the structure and function of the small intestine.                                          │
│          * Explain how nutrients are absorbed from the lumen into the bloodstream.                              │
│          * Discuss the role of villi, microvilli, and t

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2906f991-cd96-49ca-9b97-b40010463c78                                                                     │
│  Agent: Curriculum Designer                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Subject Material Researcher                                                                      │
│                                                                                                                 │
│  Task: Budi has requested a detailed, informative material on the topic of Human Digestive System.              │
│                                                                                                                 │
│  You, as a Senior Subject Material Researcher, are responsible for creating a more detailed material based on   │
│  the guideline designed by the Curriculum Designer.                                                             │
│  The material should cover all the key aspects of the topic and provide clear explanations, examples.At this    │
│  stage you dont need to do any external research or provide material references, just create the material with  │
│  all that you currently know regarding the Human Digestive System Your goal is to ensure that the material is   │
│  comprehensive, accurate, and fits the learning path outlined by the Curriculum Designer.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educational Assessor                                                                                    │
│                                                                                                                 │
│  Task: Budi has requested that you assess their understanding of the topic Human Digestive System.              │
│                                                                                                                 │
│  Your task is to create a set of exactly 5 questions based on the material provided by the Material             │
│  Researcher. These questions should range in difficulty from basic to advanced and cover all key concepts from  │
│  the material. The questions can be in the form of multiple choice, short answer, or applied questions. For     │
│  each question, also provide the correct answer to guide Budi's learning and as for testing Budi understanding  │
│  of the material.                                                                                               │
│                                                                                                                 │
│  After generating the questions, provide clear and targeted feedback for further study, including suggestions   │
│  for exercises, additional references, or areas to focus on to improve understanding of the topic.              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educational Assessor                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Your final answer must be the great and the most complete as possible, it must be outcome described.           │
│                                                                                                                 │
│  **Assessment Questions for Budi**                                                                              │
│                                                                                                                 │
│  ### Question 1: Multiple Choice (Basic)                                                                        │
│  What is the primary function of the digestive system?                                                          │
│                                                                                                                 │
│  A) To absorb water from food                                                                                   │
│  B) To break down nutrients into smaller molecules                                                              │
│  C) To eliminate waste from the body                                                                            │
│  D) To regulate body temperature                                                                                │
│                                                                                                                 │
│  Correct Answer: B) To break down nutrients into smaller molecules                                              │
│                                                                                                                 │
│  ### Question 2: Short Answer (Basic)                                                                           │
│  Describe the structure and function of the mouth in the digestive system.                                      │
│                                                                                                                 │
│  Correct Answer:                                                                                                │
│  The mouth is responsible for mechanical breakdown of food through chewing and salivary enzymes. It contains    │
│  teeth that crush and grind food, and salivary glands that produce enzymes to break down carbohydrates into     │
│  simpler sugars.                                                                                                │
│                                                                                                                 │
│  ### Question 3: Applied Question (Intermediate)                                                                │
│  A person has difficulty swallowing due to a neurological disorder. What would be the most likely consequence   │
│  on their digestive system?                                                                                     │
│                                                                                                                 │
│  Correct Answer:                                                                                                │
│  The individual may experience dysphagia, which can lead to malnutrition and weight loss due to reduced food    │
│  intake. Additionally, the digestive system may be affected by the inability to properly break down food in     │
│  the stomach.                                          

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6a503b5f-32ce-4a4b-af87-f24ea95d39b9                                                                     │
│  Agent: Educational Assessor                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b88c0295-0594-4a5d-8cda-b19af4bf0726                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described.                                                                                                     │
│                                                                                                                 │
│  **Assessment Questions for Budi**                                                                              │
│                                                                                                                 │
│  ### Question 1: Multiple Choice (Basic)                                                                        │
│  What is the primary function of the digestive system?                                                          │
│                                                                                                                 │
│  A) To absorb water from food                                                                                   │
│  B) To break down nutrients into smaller molecules                                                              │
│  C) To eliminate waste from the body                                                                            │
│  D) To regulate body temperature                                                                                │
│                                                                                                                 │
│  Correct Answer: B) To break down nutrients into smaller molecules                                              │
│                                                                                                                 │
│  ### Question 2: Short Answer (Basic)                                                                           │
│  Describe the structure and function of the mouth in the digestive system.                                      │
│                                                                                                                 │
│  Correct Answer:                                                                                                │
│  The mouth is responsible for mechanical breakdown of food through chewing and salivary enzymes. It contains    │
│  teeth that crush and grind food, and salivary glands that produce enzymes to break down carbohydrates into     │
│  simpler sugars.                                                                                                │
│                                                                                                                 │
│  ### Question 3: Applied Question (Intermediate)                                                                │
│  A person has difficulty swallowing due to a neurological disorder. What would be the most likely consequence   │
│  on their digestive system?                                                                                     │
│                                                                                                                 │
│  Correct Answer:                                                                                                │
│  The individual may experience dysphagia, which can le